<a href="https://colab.research.google.com/github/BlackBoyZeus/DataWiz/blob/main/dataprocessair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!unzip -o /content/drive/MyDrive/unet/air.zip -d /content/drive/MyDrive/unet



Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/unet/masks/kech_50400_24800_jpg.rf.9584450ee3d16f4a2e3a1fd764ed968b.jpg  
  inflating: /content/drive/MyDrive/unet/masks/kech_50400_24800_jpg.rf.95d98739712188023ea01b625696589a.jpg  
  inflating: /content/drive/MyDrive/unet/masks/kech_50400_24800_jpg.rf.96073d1633480c1db4ec51e737d5c1ad.jpg  
  inflating: /content/drive/MyDrive/unet/masks/kech_50400_24800_jpg.rf.961476a64e47d79be45f3ac32fbd28e0.jpg  
  inflating: /content/drive/MyDrive/unet/masks/kech_50400_24800_jpg.rf.968682643f4d3127f4201681e1aad7da.jpg  
  inflating: /content/drive/MyDrive/unet/masks/kech_50400_24800_jpg.rf.96b00ca9dc12c7c5b45d75289e243299.jpg  
  inflating: /content/drive/MyDrive/unet/masks/kech_50400_24800_jpg.rf.97eafca95122c67199d6fd7e6274f263.jpg  
  inflating: /content/drive/MyDrive/unet/masks/kech_50400_24800_jpg.rf.9f1b1055f7512e67e94269a83f0c5953.jpg  
  inflating: /content/drive/MyDrive/unet/masks/kech_50400_24800_jpg.r

In [4]:
def split_dataset(images, masks, split=0.2):
    split_size = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=1)
    train_y, valid_y = train_test_split(masks, test_size=split_size, random_state=1)

    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=1)
    train_y, test_y = train_test_split(train_y, test_size=split_size, random_state=1)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

In [5]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [6]:
def save_dataset(images, masks, save_dir):
    for x, y in tqdm(zip(images, masks), total=len(images)):
        name = x.split("/")[-1]

        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = cv2.imread(y, cv2.IMREAD_COLOR)

        save_image_path = os.path.join(save_dir, "images", name)
        save_mask_path = os.path.join(save_dir, "masks", name)

        cv2.imwrite(save_image_path, x)
        cv2.imwrite(save_mask_path, y)

In [7]:
dataset_path = "/content/drive/MyDrive/unet"

In [8]:
def load_dataset(path):
    images = sorted(glob(os.path.join(path, "images", "*")))
    masks = sorted(glob(os.path.join(path, "masks", "*")))
    return images, masks

In [9]:
images, masks = load_dataset(dataset_path)
print(f"Images: {len(images)} - Masks: {len(masks)}")

Images: 23879 - Masks: 23645


In [10]:
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = split_dataset(images, masks, split=0.2)



In [11]:
print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")
print(f"Test : {len(test_x)} - {len(test_y)}")

Train: 14329 - 14095
Valid: 4775 - 4775
Test : 4775 - 4775


In [12]:
save_dir = os.path.join("/content/drive/MyDrive/UnetAirplane", "aug")
for item in ["train", "valid", "test"]:
    create_dir(os.path.join(save_dir, item, "images"))
    create_dir(os.path.join(save_dir, item, "masks"))

In [13]:
%cd /content/drive/MyDrive/UnetAirplane

/content/drive/MyDrive/UnetAirplane


In [27]:
!ls

aug  files


In [15]:
save_dataset(train_x, train_y, os.path.join(save_dir, "train"))

 98%|█████████▊| 14095/14329 [11:20<00:11, 20.71it/s]


In [16]:
save_dataset(valid_x, valid_y, os.path.join(save_dir, "valid"))

100%|██████████| 4775/4775 [26:30<00:00,  3.00it/s]


In [17]:
save_dataset(test_x, test_y, os.path.join(save_dir, "test"))

100%|██████████| 4775/4775 [28:05<00:00,  2.83it/s]
